In [78]:
import os, math, random, json
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import torch
import torch.nn as nn

# >>> 项目根目录
PROJECT_ROOT = "/mnt/sda/sijiali/GlaucomaCode"
import sys
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)


from utils import get_imagenet_transform, _plot_quad
from train import get_dataloader_Salsa, SalsaHGFAlignedDataset
from dino_model import DualDinoV3LateFusion52  
from train import _make_split_ids


# 1. Config

In [79]:
# model
backbone      = "dinov3"                                 # 仅支持 dinov3
dinov3_model  = "/mnt/sda/sijiali/GlaucomaCode/pretrained_weight/dinov3-vitb16-pretrain-lvd1689m"
fusion        = "attn"                                  # "concat" / "gated-sum" / "sum" / "attn"
vit_pool      = "cls"                                     # "cls" 或 "mean_patch"
train_scope   = "all"                                    # "head" 或 "all"

if fusion == "concat":
    ckpt_path = "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_concat_lr5e-5/ckpts/best_model_epoch_302_rmse_1.0466.pth" 
elif fusion == "gated-sum":
    ckpt_path = "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_gated-sum_lr5e-5/ckpts/best_model_epoch_379_rmse_1.0206.pth"
elif fusion == "sum":
    ckpt_path = "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_sum_lr1e-4/ckpts/best_model_epoch_288_rmse_0.9195.pth"
elif fusion == "attn":
    ckpt_path = "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_attn_lr1e-4/ckpts/best_model_epoch_422_rmse_1.0707.pth"
else:
    raise ValueError(f"未知 fusion: {fusion}")
# "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_attn_lr1e-4/ckpts/best_model_epoch_422_rmse_1.0707.pth"
# "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_sum_lr1e-4/ckpts/best_model_epoch_288_rmse_0.9195.pth"
# "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_gated-sum_lr5e-5/ckpts/best_model_epoch_379_rmse_1.0206.pth"
# "/mnt/sda/sijiali/GlaucomaCode/Results_SALSA_multi/dinov3/dinov3_imagenet_all_cls_concat_lr5e-5/ckpts/best_model_epoch_302_rmse_1.0466.pth"   # 可视化的最优权重

# data
transform     = "imagenet"                                # "vit"/"cnn"/"albumentations"/"none"/"imagenet"
modality_type = "rnflt+slab"                             # "rnflt" 或 "rnflt+slab"
img_size      = 224                                       # 输入尺寸
data_root     = "/mnt/sda/sijiali/DataSet/harvardGF_unpacked"
hgf_test_root = "/mnt/sda/sijiali/DataSet/Harvard-GF/Dataset/Test"
val_ratio    = 0.1            # 仅当没有 split_txt 时用于临时划分
split_txt    = None           # 若你有固定验证集ID文件，填路径；否则保持 None
VFSCALE = 10

# 选样策略
n_random     = 6            # 从验证集随机挑 n 个样本
specified_ids= []             # 或者给定 ["data_2401", "data_1234"]，留空表示不用

# Grad-CAM 设置
target_index = None   # 例如 10 表示第 11 个 VF 点；None 表示对 52 维取和

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [80]:
out_basedir = Path("./Results_visualization/gradcam_out") 
out_dir = out_basedir / (ckpt_path.split("/")[-3] + "_" + ckpt_path.split("/")[-1].replace(".pth",""))
out_dir.mkdir(exist_ok=True, parents=True)

# 2. Build validation set & DataLoader

In [81]:
train_ids, val_ids = _make_split_ids(data_root, val_ratio=val_ratio, split_txt=split_txt, seed=42)
print(f"split => train={len(train_ids)}, val={len(val_ids)}")

# 选取要可视化的样本 ID 列表
if specified_ids:
    chosen_ids = [sid for sid in specified_ids if sid in val_ids]
else:
    rng = random.Random(2025)
    chosen_ids = rng.sample(val_ids, k=min(n_random, len(val_ids)))

print("Chosen IDs:", chosen_ids)

# transform：与训练一致
base_transform = get_imagenet_transform(img_size, with_slab=(modality_type=='rnflt+slab'))

# val 子集（通过 ids 精确采样）
val_loader = get_dataloader_Salsa(
    image_root=data_root,
    hgf_test_root=hgf_test_root,
    modality_type=modality_type,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    transform=base_transform,
    ids=chosen_ids
)


split => train=627, val=69
Chosen IDs: ['data_2553', 'data_3182', 'data_2658', 'data_2403', 'data_3052', 'data_3068']


# 3. Build model & load trained weight

In [82]:
model = DualDinoV3LateFusion52(
    rnflt_model_name=dinov3_model,
    rnflt_vit_pool=vit_pool,
    slab_model_name=dinov3_model,
    slab_vit_pool=vit_pool,
    fusion=fusion,
    head_hidden_dim=512,
    head_dropout=0.1,
    out_dim=52
).to(device)

# 加载训练权重
def load_ckpt_strict_flexible(model, ckpt_path):
    if ckpt_path is None or not os.path.exists(ckpt_path):
        print("No ckpt loaded.")
        return
    state = torch.load(ckpt_path, map_location=device)
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    # 去掉可能的 "module." 前缀
    new_state = {}
    for k, v in state.items():
        new_state[k.replace("module.", "")] = v
    missing, unexpected = model.load_state_dict(new_state, strict=False)
    print("[ckpt] loaded. missing:", missing, "unexpected:", unexpected)

load_ckpt_strict_flexible(model, ckpt_path)
model.eval()


[ckpt] loaded. missing: [] unexpected: []


DualDinoV3LateFusion52(
  (backbone_r): DinoV3FeatureBackbone(
    (net): DINOv3ViTModel(
      (embeddings): DINOv3ViTEmbeddings(
        (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (rope_embeddings): DINOv3ViTRopePositionEmbedding()
      (layer): ModuleList(
        (0-11): 12 x DINOv3ViTLayer(
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attention): DINOv3ViTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (o_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_scale1): DINOv3ViTLayerScale()
          (drop_path): Identity()
          (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): DINOv3ViTMLP(
            (up_proj): Linear(in_features=

# 4. Grad-CAM helpers

In [83]:
from pytorch_grad_cam import GradCAM, LayerCAM, GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 目标函数：index=None 表示所有 52 维的均值；否则指定某个 VF 点索引
class VFPointTarget(ClassifierOutputTarget):
    def __init__(self, index=None):
        self.index = index
    def __call__(self, model_output):
        if self.index is None:
            return model_output.sum()
        return model_output[:, int(self.index)]

# ViT 的 token -> [C,H,W] reshape（去掉 CLS 与 register tokens）
def make_vit_reshape_transform(num_register_tokens=0):
    def _rt(x):
        if isinstance(x, (list, tuple)): x = x[0]
        B, T, C = x.shape
        start = 1 + int(num_register_tokens)  # 跳过 CLS(+reg)
        N = T - start
        S = int(round(N ** 0.5))             # 期待 14
        assert S*S == N, f"patch tokens={N} 不是正方形"
        return x[:, start:, :].reshape(B, S, S, C).permute(0,3,1,2).contiguous()
    return _rt

# 自动定位 HF ViT 中“最后一个可 hook 的层”（一般用最后 block 的 LayerNorm）
def find_vit_last_layernorm(vit_model):
    """
    尝试在 HF DINOv3 ViT 里定位“最后一个可 hook 的 LN/Norm 层”
    按以下优先级：
      1) .vision_model.encoder.layers[-1].layernorm_before / .layer_norm1
      2) .vit.encoder.layers[-1].layernorm_before / .layer_norm1
      3) .encoder.layers[-1].layernorm_before / .layer_norm1
      4) .blocks[-1].norm1
      5) 常见别名 'ln'/'norm'/'layernorm'/'ln1'/'norm1'/'pre_norm'
    """
    cand_roots = []
    for name in ["vision_model", "vit", "encoder", ""]:
        node = getattr(vit_model, name, None) if name else vit_model
        if node is not None:
            cand_roots.append(node)

    for root in cand_roots:
        # encoder.layers[-1]
        enc = getattr(root, "encoder", None)
        layers = getattr(enc, "layers", None) if enc is not None else None
        if isinstance(layers, (list, tuple)) and len(layers) > 0:
            last = layers[-1]
            for nm in ["layernorm_before", "layer_norm1", "ln1", "norm1", "pre_norm"]:
                if hasattr(last, nm):
                    return getattr(last, nm)

    # 退而求其次：blocks[-1].norm1
    if hasattr(vit_model, "blocks") and isinstance(vit_model.blocks, (list, tuple)) and len(vit_model.blocks) > 0:
        last = vit_model.blocks[-1]
        for nm in ["norm1", "ln1", "layernorm_before", "pre_norm"]:
            if hasattr(last, nm):
                return getattr(last, nm)

    # 最后再试顶层别名
    for nm in ["ln", "norm", "layernorm", "ln1", "norm1", "pre_norm"]:
        if hasattr(vit_model, nm):
            return getattr(vit_model, nm)

    raise RuntimeError("Cannot locate a ViT LayerNorm to hook for Grad-CAM.")

# numpy 可视化底图（把 [1,3,H,W] 归一化到 0-1 RGB）
def to_numpy_rgb01(t):
    if t.ndim == 4:
        t = t[0]
    arr = t.detach().float().cpu().permute(1,2,0).numpy()
    arr = (arr - arr.min()) / max(arr.max() - arr.min(), 1e-6)
    return arr

# 包装器：把另一模态固定，Grad-CAM 按单输入驱动
class DualWrapperRNFLT(nn.Module):
    def __init__(self, core, fixed_slab):
        super().__init__()
        self.core = core
        self.fixed_slab = fixed_slab
    def forward(self, x):
        return self.core(x, self.fixed_slab)

class DualWrapperSLAB(nn.Module):
    def __init__(self, core, fixed_rnflt):
        super().__init__()
        self.core = core
        self.fixed_rnflt = fixed_rnflt
    def forward(self, x):
        return self.core(self.fixed_rnflt, x)


# 5. 跑 Grad-CAM（随机 n 个或指定 ID 集合）

In [84]:
target = VFPointTarget(index=target_index)

# 选择 ViT 的目标层（各自分支）
target_layer_r = model.backbone_r.net.layer[-1].norm1
target_layer_s = model.backbone_s.net.layer[-1].norm1
# 注意力层（有时更亮）
# target_layer_r = model.backbone_r.net.layer[-1].attention
# target_layer_s = model.backbone_s.net.layer[-1].attention

# ViT reshape（去掉 CLS + register）
num_reg = int(getattr(model.backbone_r, "num_register_tokens", 0))
reshape_fn = make_vit_reshape_transform(num_register_tokens=num_reg)

def _make_cam(model_wrapper, target_layers, reshape_transform, use_cuda):
    """返回一个可作为 with 使用的 GradCAM 对象，兼容不同版本"""
    try:
        return GradCAM(model=model_wrapper,
                       target_layers=target_layers,
                       reshape_transform=reshape_transform,
                       use_cuda=use_cuda)
    except TypeError:
        return GradCAM(model=model_wrapper,
                       target_layers=target_layers,
                       reshape_transform=reshape_transform)

use_cuda = (getattr(device, "type", str(device)) == "cuda")

# ========= 逆 ImageNet 归一化，用于更真实底图 =========
IM_MEAN = np.array([0.485, 0.456, 0.406]).reshape(1,1,3)
IM_STD  = np.array([0.229, 0.224, 0.225]).reshape(1,1,3)
def de_norm_imagenet_to01(t):
    if t.ndim == 4:
        t = t[0]
    arr = t.detach().float().cpu().permute(1,2,0).numpy()
    arr = arr * IM_STD + IM_MEAN
    return np.clip(arr, 0.0, 1.0)

def _np01_to_u8rgb(arr01: np.ndarray) -> np.ndarray:
    return np.clip(arr01 * 255.0, 0, 255).astype(np.uint8)

def _label_img(pil_img: Image.Image, text: str, fill=(255,255,255)) -> Image.Image:
    # 角标小字（无依赖字体，简单点）
    draw = ImageDraw.Draw(pil_img)
    pad = 6
    draw.rectangle([0, 0, draw.textlength(text)+2*pad, 20], fill=(0,0,0))
    draw.text((pad, 3), text, fill=fill)
    return pil_img

def _make_2x2_grid(img_tl: Image.Image, img_tr: Image.Image,
                   img_bl: Image.Image, img_br: Image.Image) -> Image.Image:
    # 所有子图尺寸统一到 TL 尺寸
    W, H = img_tl.size
    def _fit(im): 
        return im.resize((W, H), Image.BILINEAR)
    canvas = Image.new("RGB", (W*2, H*2), (0,0,0))
    canvas.paste(_fit(img_tl), (0,   0))
    canvas.paste(_fit(img_tr), (W,   0))
    canvas.paste(_fit(img_bl), (0,   H))
    canvas.paste(_fit(img_br), (W,   H))
    return canvas

def run_gradcam_on_batch(sample):
    sid = sample["id"][0] if isinstance(sample["id"], (list, tuple)) else sample["id"]

    x_r = sample.get("rnfl", sample.get("image")).to(device)
    x_s = sample.get("slab", None)
    if x_s is not None:
        x_s = x_s.to(device)

    # 1) 索引边界检查（52 点）
    if target_index is not None:
        idx = int(target_index)
        if not (0 <= idx < 52):
            print(f"[skip {sid}] target_index={idx} 越界(0..51)。")
            return sid

    # 2) 目标（建议 sum：信号更强）
    target = VFPointTarget(index=target_index)

    # 3) RNFLT 分支 —— 另一分支显式 detach，保持数值前向一致
    rnflt_base = to_numpy_rgb01(x_r)                         # [0,1] HWC
    wrapper_r = DualWrapperRNFLT(model, fixed_slab=(x_s.detach() if x_s is not None else torch.zeros_like(x_r)))
    cam_map_r = None
    try:
        with _make_cam(wrapper_r, [target_layer_r], reshape_fn, use_cuda) as cam_r:
            with torch.enable_grad():
                cam_map_r = cam_r(input_tensor=x_r, targets=[target])[0]
    except Exception as e:
        print(f"[RNFLT CAM failed @ {sid}] {type(e).__name__}: {e}")
        try:
            fallback = model.backbone_r.net.layer[-1].norm1
            with _make_cam(wrapper_r, [fallback], reshape_fn, use_cuda) as cam_r2:
                with torch.enable_grad():
                    cam_map_r = cam_r2(input_tensor=x_r, targets=[target])[0]
        except Exception as e2:
            print(f"[RNFLT fallback failed @ {sid}] {type(e2).__name__}: {e2}")

    # 4) SLAB 分支 —— 同理
    slab_base = to_numpy_rgb01(x_s) if x_s is not None else None
    cam_map_s = None
    if x_s is not None:
        wrapper_s = DualWrapperSLAB(model, fixed_rnflt=x_r.detach())
        try:
            with _make_cam(wrapper_s, [target_layer_s], reshape_fn, use_cuda) as cam_s:
                with torch.enable_grad():
                    cam_map_s = cam_s(input_tensor=x_s, targets=[target])[0]
        except Exception as e:
            print(f"[SLAB CAM failed @ {sid}] {type(e).__name__}: {e}")

    # 5) 把四张图拼一张（若缺 slab，则用纯黑占位）
    # RNFLT原图
    rnflt_img = Image.fromarray(_np01_to_u8rgb(rnflt_base))
    _label_img(rnflt_img, "RNFL Image")

    # RNFLT热力叠加
    if cam_map_r is not None:
        vis_r = show_cam_on_image(rnflt_base, cam_map_r, use_rgb=True)  # uint8 RGB
        rnflt_cam = Image.fromarray(vis_r)
        _label_img(rnflt_cam, f"RNFL CAM ({'idx '+str(target_index) if target_index is not None else 'sum'})")
    else:
        rnflt_cam = Image.new("RGB", rnflt_img.size, (0,0,0))
        _label_img(rnflt_cam, "RNFL CAM (N/A)")

    # SLAB原图
    if slab_base is not None:
        slab_img = Image.fromarray(_np01_to_u8rgb(slab_base))
        _label_img(slab_img, "SLAB Image")
    else:
        slab_img = Image.new("RGB", rnflt_img.size, (0,0,0))
        _label_img(slab_img, "SLAB Image (N/A)")

    # SLAB热力叠加
    if cam_map_s is not None and slab_base is not None:
        vis_s = show_cam_on_image(slab_base, cam_map_s, use_rgb=True)
        slab_cam = Image.fromarray(vis_s)
        _label_img(slab_cam, f"SLAB CAM ({'idx '+str(target_index) if target_index is not None else 'sum'})")
    else:
        slab_cam = Image.new("RGB", rnflt_img.size, (0,0,0))
        _label_img(slab_cam, "SLAB CAM (N/A)")

    # 2x2 拼图：左上 RNFLT 原图，右上 RNFLT CAM，左下 SLAB 原图，右下 SLAB CAM
    grid = _make_2x2_grid(rnflt_img, rnflt_cam, slab_img, slab_cam)
    save_name = out_dir / f"{sid}_combo_idx{target_index if target_index is not None else 'sum'}.png"
    grid.save(save_name)

    # 同时保留你原来分开存的文件（如果不再需要可删除以下三段）
    # Image.fromarray(show_cam_on_image(rnflt_base, cam_map_r, use_rgb=True)).save(out_dir / f"{sid}_gradcam_rnflt_idx{target_index if target_index is not None else 'sum'}.png")  # 可注释掉
    # if cam_map_s is not None and slab_base is not None:
    #     Image.fromarray(show_cam_on_image(slab_base, cam_map_s, use_rgb=True)).save(out_dir / f"{sid}_gradcam_slab_idx{target_index if target_index is not None else 'sum'}.png")

    print(f"[saved] {save_name}")
    
    # =========================
    # 额外保存：四联图（RNFLT, SLAB, GT, Prediction），Prediction 标题展示 MAE/RMAE
    # 计算口径严格参考你之前的代码：用 numpy 对单样本做 mae/rmse
    # =========================
    try:
        with torch.no_grad():
            # 正常双路前向，得到 (1,52)
            if x_s is not None:
                out_scaled = model(rnflt=x_r, slab=x_s)           # 网络输出（训练刻度）
            else:
                out_scaled = model(rnflt=x_r, slab=torch.zeros_like(x_r))

            out = (out_scaled * VFSCALE).detach().cpu().numpy()     # 还原到真实单位；形状 (1,52)
            pred_52 = out.reshape(-1)                              # (52,)

        # 拿 GT；没有就填 NaN
        if "label" in sample and sample["label"] is not None:
            gt_52 = sample["label"].detach().cpu().numpy().reshape(-1)
        else:
            gt_52 = np.full_like(pred_52, np.nan, dtype=np.float32)

        # —— 指标：严格按你给的口径 —— #
        if not np.isnan(gt_52).all():
            diff  = pred_52 - gt_52
            mae   = float(np.mean(np.abs(diff)))
            rmse  = float(np.sqrt(np.mean(diff ** 2)))
        else:
            mae = rmse = float("nan")

        # Prediction 标题：显示 RMAE 和 MAE
        pred_title = f"Prediction  (RMSE={rmse:.3f}, MAE={mae:.3f})" if np.isfinite(mae) else "Prediction"

        # 使用你现有的四联图函数保存（函数内部会做反标准化/绘制 52 点）
        quad_path = out_dir / f"{sid}_quad_with_metrics.png"
        _plot_quad(
            rnflt_3chw=x_r[0],                                         # (3,H,W) tensor
            slab_3chw=(x_s[0] if x_s is not None else torch.zeros_like(x_r[0])),
            gt_52=gt_52,                                               # numpy (52,)
            pred_52=pred_52,                                           # numpy (52,)
            titles=('RNFLT', 'SLAB', 'Ground truth', pred_title),
            save_path=str(quad_path)
        )
        print(f"[saved quad] {quad_path}")

    except Exception as e:
        print(f"[quad save failed @ {sid}] {type(e).__name__}: {e}")

        
    return sid


# 迭代 DataLoader（这里就是 chosen_ids 的顺序）
done = []
for sample in val_loader:
    sid = run_gradcam_on_batch(sample)
    done.append(sid)

print("Saved to:", out_dir.resolve())
print("Done IDs:", done)


Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


[saved] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2401_combo_idxsum.png
[saved quad] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2401_quad_with_metrics.png


Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


[saved] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2402_combo_idxsum.png
[saved quad] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2402_quad_with_metrics.png


Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


...

[saved] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2404_combo_idxsum.png
[saved quad] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2404_quad_with_metrics.png


Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


[saved] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2405_combo_idxsum.png
[saved quad] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2405_quad_with_metrics.png


Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Exception ignored in: <function BaseCAM.__del__ at 0x7fbf8149eee0>
Traceback (most recent call last):
  File "/mnt/sda/sijiali/anaconda3/envs/glaucoma/lib/python3.9/site-packages/pytorch_grad_cam/base_cam.py", line 212, in __del__
    self.activations_and_grads.release()
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


[saved] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2406_combo_idxsum.png
[saved quad] Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707/data_2406_quad_with_metrics.png
Saved to: /mnt/sda/sijiali/GlaucomaCode/Results_visualization/gradcam_out/dinov3_imagenet_all_cls_attn_lr1e-4_best_model_epoch_422_rmse_1.0707
Done IDs: ['data_2401', 'data_2402', 'data_2403', 'data_2404', 'data_2405', 'data_2406']
